In [1]:
import pandas as pd
import os
labels = {'pos':1, 'neg':0}
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = r"D:\fer\6. semestar\ZAVRAD\dataset\aclImdb_v1.tar\aclImdb_v1\aclImdb\%s\%s" % (s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r', encoding="utf-8") as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
df.columns = ['review', 'sentiment']

In [2]:
import numpy as np
df = df.reindex(np.random.permutation(df.index))
df.to_csv('./movie_data.csv', index=False, encoding='utf-8')

In [1]:
from nltk.corpus import wordnet
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

import nltk

from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize(tokens):
    tags = nltk.pos_tag(tokens)
    return [lemmatizer.lemmatize(word[0], get_wordnet_pos(word[1])) for word in tags]

In [3]:
import re
import string
negation_words = {'never', 'no', 'nothing', 'nowhere', 'noone', 'none', 'not', 'havent', 'hasnt', 'hadnt', 'cant',
                  'couldnt', 'shouldnt', 'wont', 'wouldnt', 'dont', 'doesnt', 'didnt', 'isnt', 'arent', 'aint', 'n\'t', 
                  "haven't", "hasn't", "hadn't", "can't", "couldn't", "shouldn't", "won't", "wouldn't", "don't", "didn't", 
                  "isn't", "aren't", "ain't", "neither", "nor"}

negate_re = re.compile(r'(' + '|'.join(negation_words) + ')', re.VERBOSE | re.I | re.UNICODE)

punctuation = set(string.punctuation)


def negate(tokens):
    words = list(tokens)
    negate_mod = False
    for i in range(len(words)):
        word = words[i]
        if negate_re.match(word):
            negate_mod = True
            continue
        elif word in punctuation:
            negate_mod = False
            continue
        if negate_mod:
            words[i] = word + "_NEG"
    return words

In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv('./movie_data.csv', encoding='utf-8')

end = 35000

X_train = df.loc[:end, 'review'].values
y_train = df.loc[:end, 'sentiment'].values
X_test = df.loc[end:, 'review'].values
y_test = df.loc[end:, 'sentiment'].values

from nltk.corpus import stopwords
stop = stopwords.words('english') + list(punctuation)

from Tokenizer import Tokenizer
Tok = Tokenizer(True, False, False)
def tok(text):
    tokens = Tok.tokenize(text)
    words = lemmatize(tokens)
    return negate(words)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(tokenizer=tok, stop_words=stop, max_features=20000, ngram_range=(1,3))
in_train = v.fit_transform(X_train).toarray()
in_test = v.transform(X_test).toarray()

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer(tokenizer=tok, stop_words=stop, max_features=20000, ngram_range=(1,3), binary=True)
in_train = v.fit_transform(X_train).toarray()
in_test = v.transform(X_test).toarray()

In [6]:
from sklearn.svm import LinearSVC
svm = LinearSVC()
svm.fit(in_train, y_train)
print(svm.score(in_test, y_test))

0.903


In [ ]:
import SVM
svm = SVM.LinearSVM()
svm.fit(in_train, y_train * 2 - 1)
print(svm.score(in_test, y_test * 2 - 1))

In [9]:
from sklearn.metrics import f1_score
f1_score(y_test * 2 - 1, svm.predict(in_test))

0.86845702339792541

In [7]:
file = open('test.txt', 'r')
test = file.read()
file.close()
print(svm.predict(v.transform([test]).toarray()))

[1]
